# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
import math
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
import matplotlib.pyplot as plt


In [31]:
data = pd.read_csv('/datasets/autos.csv') 
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


In [33]:
data['Power'].describe()

count    354369.000000
mean        110.094337
std         189.850405
min           0.000000
25%          69.000000
50%         105.000000
75%         143.000000
max       20000.000000
Name: Power, dtype: float64

In [34]:
data.query('Power>=600')

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
1816,2016-03-22 20:52:00,3200,small,2004,manual,1398,corolla,5000,6,petrol,toyota,no,2016-03-22 00:00:00,0,22043,2016-03-22 21:43:26
2102,2016-03-21 11:55:22,0,sedan,1999,NaN,1799,vectra,150000,1,petrol,opel,yes,2016-03-21 00:00:00,0,1723,2016-04-04 04:49:06
3746,2016-03-21 14:48:31,0,NaN,2017,manual,750,NaN,150000,8,petrol,smart,no,2016-03-21 00:00:00,0,49356,2016-03-24 03:44:59
4060,2016-04-03 20:31:00,3100,sedan,2005,manual,953,colt,150000,4,gasoline,mitsubishi,no,2016-04-03 00:00:00,0,60326,2016-04-07 14:56:46
5328,2016-03-29 19:44:48,500,wagon,1999,manual,1001,astra,150000,7,petrol,opel,NaN,2016-03-29 00:00:00,0,33154,2016-04-06 05:44:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348968,2016-04-04 18:53:25,250,small,1999,manual,1241,ypsilon,150000,5,petrol,lancia,yes,2016-04-04 00:00:00,0,28259,2016-04-04 18:53:25
351947,2016-03-07 21:36:19,1500,bus,2001,manual,1001,zafira,5000,7,gasoline,opel,no,2016-03-07 00:00:00,0,66117,2016-03-09 12:47:08
353493,2016-04-02 20:54:21,12500,NaN,2017,manual,2000,other,60000,0,gasoline,chrysler,no,2016-04-02 00:00:00,0,44145,2016-04-06 21:44:39
353633,2016-03-23 23:55:21,2400,sedan,2007,manual,650,c2,150000,8,petrol,citroen,NaN,2016-03-23 00:00:00,0,45277,2016-03-27 01:15:17


В дата сете есть пропуски, также есть аномальные данные, например в столбце Power, видимо ошибка при занесении

In [35]:
data['Brand'].isnull().sum()

0

In [36]:
data[['Price','Brand','VehicleType']]

,Price,Brand,VehicleType
0,480,volkswagen,NaN
1,18300,audi,coupe
2,9800,jeep,suv
3,1500,volkswagen,small
4,3600,skoda,small
...,...,...,...
354364,0,mitsubishi,NaN
354365,2200,sonstige_autos,NaN
354366,1199,smart,convertible
354367,9200,volkswagen,bus


In [37]:
chek = data.pivot_table('Price',index=['Brand','VehicleType'])
chek

Price
Brand      VehicleType              
alfa_romeo convertible   7035.727778
           coupe         5106.205240
           other         3153.333333
           sedan         3045.095238
           small         4466.242678
...                              ...
volvo      other        11588.000000
           sedan         4680.300156
           small         3912.714286
           suv          11857.031674
           wagon         3247.175968

[296 rows x 1 columns]

In [39]:
data['Price'].replace('0', chek)

0           480
1         18300
2          9800
3          1500
4          3600
          ...  
354364        0
354365     2200
354366     1199
354367     9200
354368     3400
Name: Price, Length: 354369, dtype: int64

In [44]:
data

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,sedan,1993,manual,0,golf,150000,0,petrol,volkswagen,no,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,golf,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,no,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,2016-03-21 09:50:58,0,sedan,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,2016-03-21 00:00:00,0,2694,2016-03-21 10:42:49
354365,2016-03-14 17:48:27,2200,sedan,2005,manual,0,golf,20000,1,petrol,sonstige_autos,no,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
354366,2016-03-05 19:56:21,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
354367,2016-03-19 18:57:12,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26


In [45]:
data["VehicleType"] = data["VehicleType"].fillna(data["VehicleType"].mode().values[0])
data["Gearbox"] = data["Gearbox"].fillna(data["Gearbox"].mode().values[0])
data["Model"] = data["Model"].fillna(data["Model"].mode().values[0])
data["FuelType"] = data["FuelType"].fillna(data["FuelType"].mode().values[0])

data["NotRepaired"] = data["NotRepaired"].fillna("no")

In [46]:
#data["VehicleType"].describe()

objects_columns = ["VehicleType","Gearbox","Model","FuelType","Brand","NotRepaired"]
for column in objects_columns:
    print(column, pd.Series(data[column].unique()).str.lower().duplicated().sum())

In [47]:
data.duplicated().sum()

5

In [48]:
data = data.drop_duplicates()
data.duplicated().sum()

0

In [49]:
#print(data["Model"].fillna(data["Model"].mode().values[0]))

In [50]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354364.000000,354364.000000,354364.000000,354364.000000,354364.000000,354364.0,354364.000000
mean,4416.655608,2004.234471,110.093723,128211.373051,5.714641,0.0,50508.461698
std,4514.159690,90.228593,189.851590,37905.136957,3.726433,0.0,25783.124276
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49409.500000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Удалим столбцы: которые больше не потребуются

In [51]:
data_with_date = data.copy()
data = data.drop(["DateCrawled","DateCreated","LastSeen","NumberOfPictures","PostalCode"],axis = 1)

###### Вывод:

Проанализируем значения числовых признаков перед применения к ним алгоритма очистки выбросов

Можно заметить, что в столбце RegistrationYear присутствуют аномальные значения, предлагаю установить границы, в котором может находится значение этого столбца. А именно, год регистрации не может быть больше 2020, и не может быть меньше 1930. Все показания выше или ниже этих значений приведем к максимальной или минимальной границе (признак не нуждается в поиске выбросов)

Можно заметить, что в столбце RegistrationMonth, встречается значение 0, которое корректнее было бы изменить на 1.

Так же в Power присутствуют аномальные значения (20 000 лошадиных сил), ограничим их  (признак нуждается в поиске выбросов)

Столбец Kilometer имеет достаточно реальные значения, поэтому смысла искать там выбросы нет 


Такие признаки как NumberOfPictures и PostalCode (признаки не нуждается в поиске выбросов) , так как не влияют на реальную стоимость автомобиля (их мы просто удалим)

###### Обработаем выбросы и границы значений в упомянутых выше признаках

In [52]:
# RegistrationYear 
def Balance_RegistrationYear(value):
    if value > 2020:
        return 2020
    elif value < 1990:
        return 1990
    else:
        return value
data["RegistrationYear"] = data["RegistrationYear"].apply(Balance_RegistrationYear)
# RegistrationMonth
data.loc[data['RegistrationMonth'] == 0, 'RegistrationMonth'] = 1
# Power
data.loc[data['Power'] > 600, 'Power'] = 600

In [53]:
data['Power'].describe()

count    354364.000000
mean        107.010946
std          65.475871
min           0.000000
25%          69.000000
50%         105.000000
75%         143.000000
max         600.000000
Name: Power, dtype: float64

In [55]:
data.corr()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth
Price,1.000000,0.439346,0.469027,-0.333208,0.098893
RegistrationYear,0.439346,1.000000,0.096869,-0.243613,0.033220
Power,0.469027,0.096869,1.000000,0.082051,0.114145
Kilometer,-0.333208,-0.243613,0.082051,1.000000,0.009732
RegistrationMonth,0.098893,0.033220,0.114145,0.009732,1.000000


In [56]:
#data.query('RegistrationYear <= 1930')

## Обучение моделей

###### Разделим и закодируем выборки

In [57]:
data_ohe = data.copy()
data_ohe = pd.get_dummies(data_ohe)
data_light = data.copy()
data_light = data_light.astype({"VehicleType":'category',
                                  "Gearbox":'category',
                                  "Model":'category',
                                  "FuelType":'category',
                                  "Brand":'category',
                                  "NotRepaired":'category'})

# Разделим данные на выборки

In [58]:
trainX,testX,trainY,testY = train_test_split(data_ohe.drop("Price",axis = 1),
                                             data_ohe["Price"],
                                             test_size = 0.25,
                                             random_state = 42)

trainX_wo_ohe,testX_wo_ohe,trainY_wo_ohe,testY_wo_ohe = train_test_split(data.drop("Price",axis = 1),
                                             data["Price"],
                                             test_size = 0.25,
                                             random_state = 42)
(trainX_wo_ohe_light,
 testX_wo_ohe_light,
 trainY_wo_ohe_light,
 testY_wo_ohe_light) = train_test_split(data_light.drop("Price",axis = 1),
                                             data_light["Price"],
                                             test_size = 0.25,
                                             random_state = 42)

In [ ]:
%%time
# catboost
catboost = CatBoostRegressor(loss_function='RMSE')
parameters_cat = {'depth':[5,10], 'learning_rate':np.arange(0.1,1,0.2)}
catboost_grid = catboost.grid_search(parameters_cat,
            Pool(trainX_wo_ohe, trainY_wo_ohe, cat_features=["VehicleType","Gearbox",
                                                             "Model","FuelType",
                                                             "Brand","NotRepaired"]),
            cv=3,
            verbose=True,
            plot=False)

0:	loss: 1810.2064996	best: 1810.2064996 (0)	total: 8m 13s	remaining: 1h 13m 57s
1:	loss: 1775.9454291	best: 1775.9454291 (1)	total: 16m 29s	remaining: 1h 5m 56s
2:	loss: 1786.4801386	best: 1775.9454291 (1)	total: 24m 39s	remaining: 57m 32s
3:	loss: 1800.3052889	best: 1775.9454291 (1)	total: 32m 54s	remaining: 49m 22s
4:	loss: 1825.2248696	best: 1775.9454291 (1)	total: 41m 19s	remaining: 41m 19s
5:	loss: 1736.0247078	best: 1736.0247078 (5)	total: 1h 1m 48s	remaining: 41m 12s
6:	loss: 1751.8012133	best: 1736.0247078 (5)	total: 1h 22m 30s	remaining: 35m 21s
7:	loss: 1783.9631787	best: 1736.0247078 (5)	total: 1h 42m 58s	remaining: 25m 44s


In [87]:
catboost_grid['params']

{'depth': 10, 'learning_rate': 0.1}

In [88]:
%%time
final_cat = CatBoostRegressor(depth=catboost_grid['params']['depth'], 
                              learning_rate=catboost_grid['params']['learning_rate'],
                              loss_function='RMSE',verbose=100)
final_cat.fit(Pool(trainX_wo_ohe,trainY_wo_ohe,
                   cat_features=['VehicleType','Gearbox','Model','FuelType','Brand','NotRepaired']))

0:	learn: 4196.7204848	total: 1.42s	remaining: 23m 33s
100:	learn: 1800.1696715	total: 2m 14s	remaining: 19m 53s
200:	learn: 1710.5146736	total: 4m 25s	remaining: 17m 37s
300:	learn: 1654.9959507	total: 6m 42s	remaining: 15m 34s
400:	learn: 1612.3559760	total: 9m 1s	remaining: 13m 28s
500:	learn: 1579.1563173	total: 11m 17s	remaining: 11m 14s
600:	learn: 1550.9294880	total: 13m 37s	remaining: 9m 2s
700:	learn: 1526.6763400	total: 15m 58s	remaining: 6m 48s
800:	learn: 1506.0560264	total: 18m 18s	remaining: 4m 32s
900:	learn: 1485.9453965	total: 20m 38s	remaining: 2m 16s
999:	learn: 1467.5271013	total: 23m	remaining: 0us
CPU times: user 21min 26s, sys: 1min 39s, total: 23min 5s
Wall time: 23min 8s


In [89]:
%%time
# RandomForest
RF = RandomForestRegressor(criterion = 'mse',random_state=42,n_estimators = 100,n_jobs=-1)
RF.fit(trainX,trainY)

CPU times: user 15min 17s, sys: 0 ns, total: 15min 17s
Wall time: 15min 38s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                      oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [90]:
%%time
# LightGBM
lgb_train = lgb.Dataset(trainX_wo_ohe_light, trainY_wo_ohe_light)
lgb_test = lgb.Dataset(testX_wo_ohe_light, testY_wo_ohe_light, reference=lgb_train)
parameters_light = {'metric': 'l2', 'max_depth':10,'random_state': 42,'learning_rate':0.1}
light = lgb.train(parameters_light,
                lgb_train,
                num_boost_round=1000,
                valid_sets=[lgb_train, lgb_test],
                verbose_eval=100)

[100]	training's l2: 3.11514e+06	valid_1's l2: 3.23958e+06
[200]	training's l2: 2.90792e+06	valid_1's l2: 3.12195e+06
[300]	training's l2: 2.77442e+06	valid_1's l2: 3.05903e+06
[400]	training's l2: 2.67665e+06	valid_1's l2: 3.02013e+06
[500]	training's l2: 2.5892e+06	valid_1's l2: 2.98712e+06
[600]	training's l2: 2.51952e+06	valid_1's l2: 2.96558e+06
[700]	training's l2: 2.4584e+06	valid_1's l2: 2.94495e+06
[800]	training's l2: 2.40855e+06	valid_1's l2: 2.93347e+06
[900]	training's l2: 2.36453e+06	valid_1's l2: 2.92332e+06
[1000]	training's l2: 2.32414e+06	valid_1's l2: 2.91469e+06
CPU times: user 1min 19s, sys: 0 ns, total: 1min 19s
Wall time: 1min 21s


In [91]:
%%time
print("CatBoost:",mean_squared_error(testY,final_cat.predict(testX_wo_ohe))**0.5)

CatBoost: 1711.6634987925677
CPU times: user 2.69 s, sys: 0 ns, total: 2.69 s
Wall time: 2.7 s


In [92]:
%%time
print("RandomForest:",mean_squared_error(testY,RF.predict(testX))**0.5)

RandomForest: 1759.7767496781123
CPU times: user 6.81 s, sys: 0 ns, total: 6.81 s
Wall time: 6.92 s


In [93]:
%%time
print("LightGBM:",mean_squared_error(testY,light.predict(testX_wo_ohe_light))**0.5)

LightGBM: 1707.2463775468862
CPU times: user 11.4 s, sys: 0 ns, total: 11.4 s
Wall time: 11.4 s


## Анализ моделей

В условии задачи для выбора модели у нас изначально было три условия:

1. Время предсказания меньше
2. Качество предсказаний ниже
3. Время обучения больше

Исходя из полученных результатов можо сделать вывод, что "Случайный Лес" нам сразу не подходит, т.к. у него самое большое время обучения (11 мин) и самая низкая метрика качества RMSE, " второе место"-CatBoost, самые лучшие данные по скорости обучения и метрике качества получились у LightGBM, RMSE 1695 и время обучения 1 мин.

Основываясь на этом лучше всего нам подойдет модель LightGBM.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнена загрузка и подготовка данных
- [ ]  Выполнено обучение моделей
- [ ]  Есть анализ скорости работы и качества моделей